In [4]:
import requests
from bs4 import BeautifulSoup
import re
import os.path

In [5]:
# Dog Information URL
def dog_url_list(url):
    dogpage = requests.get(url)
    lists = re.findall('<a href="(http://animal-adoption.coa.gov.tw/animal/animal_add_ctr/\?p=\d+&id=\d+\S*)"', dogpage.text)
    
    return lists


In [6]:
def dog_content(url):
    '''
      dog ={
             '備註': '',
             '區域縣市': '臺中市',
             '尋獲地點': '',
             '年紀': '幼年',
             '性別': '公',
             '所在地': '南屯園區',
             '所屬單位': '臺中市動物之家南屯園區',
             '毛色': '棕色',
             '流水編號': '10702131002007',
             '照片': 'http://animal-adoption.coa.gov.tw/uploads/animal_album/91/3a1ccbab922e0f35bac1f1ddca0d6aa4.png',
             '體型': '中型'
           }
    '''
    dog = {}
    
    #ConnectionError
    dogcontent = requests.get(url,headers={'Connection':'close'})
    soup = BeautifulSoup(dogcontent.text,'lxml')
    #soup = BeautifulSoup(dogcontent.text,'html5lib')
    
    #dog content
    dog['流水編號'] = soup.select('#printblock > p:nth-of-type(2) > span.value')[0].text
    dog['所屬單位'] = soup.select('#printblock > p:nth-of-type(4) > span.value')[0].text
    dog['所在地']   = soup.select('#printblock > p:nth-of-type(5) > span.value')[0].text
    dog['區域縣市'] = soup.select('#printblock > p:nth-of-type(6) > span.value')[0].text
    dog['性別'] = soup.select('#printblock > p:nth-of-type(7) > span:nth-of-type(4)')[0].text
    dog['體型'] = soup.select('#printblock > p:nth-of-type(8) > span.value')[0].text
    dog['毛色'] = soup.select('#printblock > p:nth-of-type(8) > span:nth-of-type(4)')[0].text
    dog['年紀'] = soup.select('#printblock > p:nth-of-type(9) > span.value')[0].text
    dog['尋獲地點'] = soup.select('#printblock > p:nth-of-type(10) > span.value ')[0].text
    dog['備註'] = soup.select('#printblock > p.remark > span.value')[0].text
    dog['時間'] = soup.select('#printblock > p:nth-of-type(2) > span.value')[0].text[0:7]
    dog['網址'] = url
    
    #if imgs's block has photo ,save it else save none in dict {}
    # 照片有 jpg & png格式
    imgs = soup.select('#print_img > div')
    try:
        #dog['照片'] = re.findall('<img src="(http://animal-adoption.coa.gov.tw/uploads/animal_album/\S*)"',dogcontent.text)[0]
        dog['照片'] = re.findall('src="(http://animal-adoption.coa.gov.tw/uploads/animal_album/\S*)"',str(imgs))[0]
    except IndexError:
        dog['照片'] = None
        
    return dog

In [7]:
#!pip install pillow
#存圖片
import shutil
import os

import requests
from PIL import Image

def save_photo(url,filename):
    try:
        resp = requests.get(url, stream=True)
        img = Image.open(resp.raw) # 把圖片load進去Pillow裡面
    
        if 'png' in url: #照片有png和jpg兩種格式
            img.save('./Missingdogphoto/{}.png'.format(filename))
        else:
            img.save('./Missingdogphoto/{}.jpg'.format(filename))
    # OSError 沒有照片
    except OSError:
        return print('no picture')

In [8]:
# 連到MONGODB
#!pip install pymongo
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
#if mongoDB has no db and collection , Create automatically.
db = client.missingdog
collection = db.dogInfo
collection.count()
#collection.remove({})

4480

In [ ]:
#塞狗狗資訊到MONGODB
import datetime as dt
start1 = dt.datetime.now()
i = 223

while True:    
    HOST = 'http://animal-adoption.coa.gov.tw/index.php/animal'
    url_page = HOST + '?s_area=ALL&s_kind=%E7%8B%97&s_bodytype=ALL&s_age=&s_color=ALL&s_sex=ALL&num=20&p={}'.format(i)
    print('page ' + str(i))
    if dog_url_list(url_page) != []:
        dogurls = dog_url_list(url_page)

        dogs = []
        
        start2 = dt.datetime.now()
        
        for url in dogurls:
            content = dog_content(url)
            save_photo(content['照片'],content['流水編號'])
            dogs.append(content)
            if content['照片'] == None:
                print(content['照片'])
        
        print(len(dogs))
        collection.insert_many(dogs)
        
        end2 = dt.datetime.now()
        print('Total time {} s.'.format((end2 - start2).seconds))
        
    else:
        break
    i +=1
end1 = dt.datetime.now()
print('Total time {} s.'.format((end1 - start1).seconds))
   
    
    

page 223
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
20
Total time 48 s.
page 224
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
None
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
20
Total time 30 s.
page 225
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
20
Total time 19 s.
page 226
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
None
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
no picture
20
Total time 